In [1]:
import surprise
import pandas as pd
import numpy as np

from surprise.model_selection import KFold
from surprise.model_selection import cross_validate

## data 스크래이핑

In [4]:
name1=['김밥천국','포보스','삼백집','밥뜨랑','다원','한려수도','봉태민','종로김밥','명동칼국수','돈토']

In [ ]:
import os
import sys
import urllib.request
import datetime
import time
import json

def get_request_url(url) :
    client_id = 'id'
    client_secret= 'Secret'
    rep = urllib.request.Request(url)
    rep.add_header("X-Naver-Client-Id",client_id)
    rep.add_header("X-Naver-Client-Secret",client_secret)
    try :
        response = urllib.request.urlopen(rep)
        if response.getcode() ==  200 :
            print("[%s] Url request success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s]Error for URL : %s" % (datetime.datetime.now(), url))
        return None

    
def getNaverSearchResult(sNode, search_text, page_start, display) :
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % (sNode)
    params = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(search_text),
                                               page_start, display)
    url = base+node+params
    getData = get_request_url(url)
    if getData == None :
        return None
    else :
        return json.loads(getData)

In [ ]:
#검색결과 json으로 저장하기 
# 원하는 검색 종류에 따라 형식 달라서
#getPostData 함수의 수정이 필요
# 이거는 blog 내용

def getPostData(post, jsonResult) :
    title = post['title']
    description = post['description']
    link = post['link']
    bloggername=post['bloggername']
    bloggerlink=post['bloggerlink']
    
    
    jsonResult.append({'title' : title, 'description' : description,
                      'link' : link, 'blogname' : bloggername,
                       'bloggerlink' : bloggerlink})
    return

def main(search_text) :
    jsonResult = [] #초기화
    
    #내 개인 설정 
    sNode = 'blog'    #'news', 'blog', 'cafearticle'
    name_len = len(name1)
    display_count = 100
    
    jsonSearch = getNaverSearchResult(sNode, search_text, name_len , display_count)
    
    while ((jsonSearch !=None) and (jsonSearch['display'] !=0)) :
        for post in jsonSearch['items'] :
            getPostData(post, jsonResult)
        
        nStart = jsonSearch['start'] + jsonSearch['display']
        jsonSearch = getNaverSearchResult(sNode, search_text, nStart, display_count)
        
    with open('%s_naver_%s.json' % (search_text, sNode), 'w', encoding='utf8') as outfile :
        retJson = json.dumps(jsonResult, indent = 4, sort_keys = True, ensure_ascii=False)
        outfile.write(retJson)   
    print('%s_naver_%s.json SAVED' % (search_text, sNode))

    

In [ ]:
if __name__ == '__main__' :
    main(name1[0])

In [ ]:
sNode = 'blog'
name_dict={}
for n,i in enumerate(name1) :
    if '+' in i :
        name_dict[i.split('+')[1].split(' ')[0]] = n
    else :
        name_dict[i] = n

In [ ]:
name_dict

In [ ]:
for i in range(name_len) :
    a = open(name1[i]+'_naver_blog.json','rt', encoding='UTF8')
    json1.append(json.load(a))
a.close()

## 데이터 전처리

In [ ]:
#blog이름에 컨설팅, 창업이 있는 description 제거

for i in range(0,11) :
    for j in range(len(json1[i])) :
        if ('컨설팅' in json1[i][j]['blogname']) | ('창업' in json1[i][j]['blogname']) :
            continue
        else :
            des.append(json1[i][j]['description'])  

In [ ]:
len(des)

In [ ]:
des1=[i.replace('<b>','').replace('</b>','') for i in des]

In [ ]:
# 저장 

In [ ]:
pd.DataFrame(des1).to_csv('lunch_project2',encoding='UTF8')

In [ ]:
#blog이름에 컨설팅, 창업이 있는 description 제거
# 각 검색어의 검색결과 개수 계산
sum=0
suma=[]
for i in range(0,11) :
    for j in range(len(json1[i])) :
        if ('컨설팅' in json1[i][j]['blogname']) | ('창업' in json1[i][j]['blogname']) :
            continue
        else :
            sum+=1
    suma.append(sum)
    sum=0

In [ ]:
np.unique

In [ ]:
sum1=0

temp1=[]
des2=[]
for i in range(11) :
    for j in range(suma[i]) :
        temp1.append(des1[sum1+j])
    des2.append(list(np.unique(temp1)))
    temp1=[]
    sum1+=suma[i]

In [ ]:
# 각 검색어의 결과 위치
suma2=[len(i) for i in des2]

In [ ]:
#okt 사용 토큰화
import nltk
from konlpy.tag import Okt

In [ ]:
def token_count(words) :
    t = Okt()
    word_dic = {}
    malist = t.pos(words, stem=True)
    for word in malist :
        if word[1] in ['Noun', 'Verb', 'Adjective'] :
            word_dic[word[0]] = word_dic.get(word[0], 0) + 1
    return word_dic

In [ ]:
#stopword 지정 - notcounting

In [ ]:
notcounting=['장애인','사업', '컨설팅', '생산', '매물', '우엉갈비', '캐드', '거가대교', '주부', '승마장', '공화국', '전남대', '방위비', '횟집', '창업','JAVA', '이마트', '종가집', '곽지민', '배달' ,'건축', '건설', '화장품', '인쇄', '보호', '작업장','에어컨', '문화', '섬유', '학교', '법인', '비밀넷', '콘트롤', '태권도', '어린이집', '예은', '휴대폰', '일본']

In [ ]:
word_dict0=token_count(des2[0])
keys = sorted(word_dict0.items(), key = lambda x : x[1], reverse = True)
for word, count in keys :
    print("{}({})".format(word, count), end="")
    print()

# 워드클라우드 시각화

In [5]:
import nltk
from konlpy.corpus import kolaw
from konlpy.tag import Okt
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
from wordcloud import WordCloud
from wordcloud import ImageColorGenerator

def token_count(words) :
    t = Okt()
    word_dic = {}
    malist = t.pos(words, stem=True)
    
def towordcl(worddict, notcounting=[]) :
    text1=[]
    aa=''
    for i in worddict :
        if i[1] in ['Noun', 'Verb', 'Adjective'] :
            text1.append(i[0])
    tokens_ko = [each_word for each_word in text1 if each_word not in notcounting]
    sel_word = nltk.Text(tokens_ko) # nltk의 Text형태로 묶는..?
    data = sel_word.vocab().most_common(1000)          
    tmp_data = dict(data)
    wordcloud=WordCloud(font_path = "C:/Windows/Fonts/YBUU02.ttf", background_color='white')
    wordcloud=wordcloud.generate_from_frequencies(tmp_data)
    plt.figure(figsize=(12,12))
    plt.imshow(wordcloud, interpolation = 'bilinear')
    plt.axis('off')
    plt.show()
    
for i in range(11) :
    t=Okt()
    malist = t.pos(des3[i], stem=True)
    towordcl(malist, notcounting)

C:\Users\Affinity\.conda\envs\keras\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


NameError: name 'des3' is not defined

## 데이터 분석

In [7]:
#dataset 모양 정리
unch2=pd.read_csv('lunch_final.csv',encoding = 'cp949')
lunch2.head(1)

NameError: name 'pd' is not defined

In [3]:
lun_daf=lunch2.iloc[:,1:11][:25]
lun_daf

,김밥천국,포베이,삼백집,밥뜨랑,다원,한려수도,봉태민,종로김밥,명동칼국수,돈토
0,3,4,3,3,3,5,3,3,4,3
1,2,4,4,3,4,3,5,4,4,3
2,1,2,4,3,3,4,3,1,3,3
3,4,5,5,5,5,5,5,4,5,4
4,2,4,3,2,4,5,3,3,5,4
5,3,3,2,3,4,4,3,4,4,4
6,4,4,3,5,5,5,5,3,2,4
7,3,4,2,5,4,4,4,2,3,4
8,2,4,4,3,3,4,5,3,5,3
9,1,4,3,2,5,1,5,2,4,5


In [4]:
#lunda.columns=['김밥천국','포보스','삼백집','밥뜨랑','다원','한려수도','봉태민','종로김밥','명동칼국수','돈토']
lun_daf.index=lunch2['이름'][:25]

In [5]:
len(lun_daf)

25

In [6]:
user=[]
for i in range(25):
        m=(np.ones(10)*i)
        m=list(m)
        user.append(m)

item=[]
p=np.arange(10)
for i in range(25):
    c=list(p)
    item.append(p)

item=np.array(item).reshape(250,)
user=np.array(user).reshape(250,)
rating=np.array(lun_daf).astype('int').reshape(250,)

In [7]:
da=np.concatenate((user.astype('str'),item,rating))

In [8]:
da=da.reshape([3,-1]).T

In [9]:
da2=pd.DataFrame(da)

In [10]:
da2.iloc[:,2] = [int(i) for i in da[:,2]]

In [11]:
da2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 3 columns):
0    250 non-null object
1    250 non-null object
2    250 non-null int64
dtypes: int64(1), object(2)
memory usage: 5.9+ KB


In [12]:
tuple(da2.iloc[0,:])

('0.0', '0', 3)

In [13]:
dat=[]
for i in range(250):
    t=tuple(da2.iloc[i,:])
    dat.append(t)

# surprise의 dataset으로 만들어주기

In [14]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader

df = pd.DataFrame(dat, columns=['user','item','rating'])
reader = Reader(rating_scale=(1, 5))
lundata= surprise.dataset.Dataset.load_from_df(df,reader)

C:\Users\Affinity\.conda\envs\keras\lib\site-packages\scikit_surprise-1.0.6-py3.6-win-amd64.egg\surprise\dataset.py:61: UserWarning: Using rating_scale from reader, deprecated. Set rating_scale at dataset creation instead (load_from_file, load_from_folds, or load_from_df).
  warnings.warn('Using rating_scale from reader, deprecated. Set '


# baseline - KFold

In [15]:
bsl_param={'method':'als','n_epochs':5, 'reg_u':7, 'reg_i':3}
bsl_param2={'method':'sgd','reg':0.02, 'learning_rate':0.005, 'n_epochs':10}

# 학습 알고리즘 
: sgd(stochastic Gradient Descent) 사용

In [16]:
np.random.seed(1)
model= surprise.BaselineOnly(bsl_options=bsl_param2)

acc=np.zeros(3)
cv=KFold(3)

for i ,(trainset, testset) in enumerate(cv.split(lundata)):
    model.fit(trainset)
    pred=model.test(testset)
    acc[i] = surprise.accuracy.mae(pred, verbose=True)
acc.mean()

Estimating biases using sgd...
MAE:  0.8354
Estimating biases using sgd...
MAE:  0.8213
Estimating biases using sgd...
MAE:  0.7744


0.8103944605580172

:Als(alternating Least Squares) 사용

In [17]:
np.random.seed(1)
model= surprise.BaselineOnly(bsl_options=bsl_param)

acc=np.zeros(3)
cv=KFold(3)

for i ,(trainset, testset) in enumerate(cv.split(lundata)):
    model.fit(trainset)
    pred=model.test(testset)
    acc[i] = surprise.accuracy.mae(pred, verbose=True)
acc.mean()

Estimating biases using als...
MAE:  0.7840
Estimating biases using als...
MAE:  0.8352
Estimating biases using als...
MAE:  0.7755


0.798215706475471

# baseline - cross_validation

In [18]:
# : sgd(stochastic Gradient Descent) 사용
np.random.seed(1)

model2= surprise.BaselineOnly(bsl_options=bsl_param2)
cross_validate(model2, lundata)
acc = surprise.accuracy.mae(pred, verbose=True)

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
MAE:  0.7755


In [19]:
#:Als(alternating Least Squares) 사용
np.random.seed(1)

model2= surprise.BaselineOnly(bsl_options=bsl_param)
cross_validate(model2, lundata)
acc = surprise.accuracy.mae(pred, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
MAE:  0.7755


# 이웃 기반 모형
: 유사도 기반

# 1. msd : 평균제곱차이 유사도

In [20]:
np.random.seed(1)

sim_options = {'name':'msd'}
algo= surprise.KNNBasic(sim_options = sim_options)
cross_validate(algo, lundata)["test_mae"].mean()

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


0.7930861289915383

# cosine 유사도

In [21]:
np.random.seed(1)

sim_options = {'name':'cosine'}
algo= surprise.KNNBasic(sim_options = sim_options)
cross_validate(algo, lundata)["test_mae"].mean()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


0.8220529014486699

# pearson 유사도
: 두 벡터의 상관 계수를 -1~1의 수로 나타냄

In [22]:
np.random.seed(1)

sim_options = {'name':'pearson'}
algo= surprise.KNNBasic(sim_options = sim_options)
cross_validate(algo, lundata)["test_mae"].mean()

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


0.7781985364593123

# pearson_baseline 유사도
: 각 벡터의 기대값을 단순 평균이 아니라 베이스라인 모형에서 예측한 값을 사용

In [23]:
np.random.seed(1)

sim_options = {'name':'pearson_baseline'}
algo= surprise.KNNBasic(sim_options = sim_options)
cross_validate(algo, lundata)["test_mae"].mean()

Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


0.7520818728670766

# 3. matrix Factorization
: 모든 사용자와 상품에 대해 오차 함수를 최소화하는 요인 벡터를 탐색

# SVD

In [24]:
np.random.seed(1)

algo= surprise.SVD(n_factors=50)
cross_validate(algo, lundata)["test_mae"].mean()

0.8038445606836927

# SVDpp

In [25]:
np.random.seed(1)

algo= surprise.SVDpp(n_factors=70)
cross_validate(algo, lundata)["test_mae"].mean()

0.792798991509627

# NMF(Non-negative matrix factorization)

In [26]:
np.random.seed(1)

algo= surprise.NMF(n_factors=100)
cross_validate(algo, lundata)["test_mae"].mean()

0.7796614502801005

# surprise의 dataset으로 만들어주기2

In [ ]:
user=[]
for i in range(25):
        m=(np.ones(10)*i)
        m=list(m)
        user.append(m)

item=[]
p=np.arange(10)
for i in range(25):
    c=list(p)
    item.append(p)

item=np.array(item).reshape(250,)
user=np.array(user).reshape(250,)
rating=np.array(lun_daf).reshape(250,)

# da=np.concatenate((user,item,rating))
# da=da.reshape([3,-1]).T #zip 한줄로 가능
da=list(zip(user,item,rating))

dat2=[] # tuple안에 list 형태로 되어있어서 형태를 만들어줌
for i in range(250):
    t=tuple(da[i])
    dat2.append(t)

In [28]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader

df2 = pd.DataFrame(dat2, columns=['user','item','rating'])
reader = Reader(rating_scale=(1, 5))
lundata2= surprise.dataset.Dataset.load_from_df(df2,reader)

In [29]:
lundata2.raw_ratings

[(0.0, 0.0, 3.0, None),
 (0.0, 1.0, 4.0, None),
 (0.0, 2.0, 3.0, None),
 (0.0, 3.0, 3.0, None),
 (0.0, 4.0, 3.0, None),
 (0.0, 5.0, 5.0, None),
 (0.0, 6.0, 3.0, None),
 (0.0, 7.0, 3.0, None),
 (0.0, 8.0, 4.0, None),
 (0.0, 9.0, 3.0, None),
 (1.0, 0.0, 2.0, None),
 (1.0, 1.0, 4.0, None),
 (1.0, 2.0, 4.0, None),
 (1.0, 3.0, 3.0, None),
 (1.0, 4.0, 4.0, None),
 (1.0, 5.0, 3.0, None),
 (1.0, 6.0, 5.0, None),
 (1.0, 7.0, 4.0, None),
 (1.0, 8.0, 4.0, None),
 (1.0, 9.0, 3.0, None),
 (2.0, 0.0, 1.0, None),
 (2.0, 1.0, 2.0, None),
 (2.0, 2.0, 4.0, None),
 (2.0, 3.0, 3.0, None),
 (2.0, 4.0, 3.0, None),
 (2.0, 5.0, 4.0, None),
 (2.0, 6.0, 3.0, None),
 (2.0, 7.0, 1.0, None),
 (2.0, 8.0, 3.0, None),
 (2.0, 9.0, 3.0, None),
 (3.0, 0.0, 4.0, None),
 (3.0, 1.0, 5.0, None),
 (3.0, 2.0, 5.0, None),
 (3.0, 3.0, 5.0, None),
 (3.0, 4.0, 5.0, None),
 (3.0, 5.0, 5.0, None),
 (3.0, 6.0, 5.0, None),
 (3.0, 7.0, 4.0, None),
 (3.0, 8.0, 5.0, None),
 (3.0, 9.0, 4.0, None),
 (4.0, 0.0, 2.0, None),
 (4.0, 1.0, 4.0,

# predict

In [30]:
data = lundata2

# Retrieve the trainset.
trainset = data.build_full_trainset()

# Build an algorithm, and train it.
sim_options = {'name':'msd'}
algo= surprise.KNNBasic(sim_options = sim_options)
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [31]:
test1=rating.copy()
test1=test1.reshape((-1,10))
test2=test1.mean(axis=0)

In [32]:
user_no=2.0

from surprise import PredictionImpossible
ests=[]
for i in range(10):
    a= test2[i]
    pred = algo.predict(user_no, i, r_ui= a, verbose=True)
    ests.append(pred.est)

user: 2.0        item: 0          r_ui = 2.56   est = 2.44   {'actual_k': 25, 'was_impossible': False}
user: 2.0        item: 1          r_ui = 3.44   est = 3.28   {'actual_k': 25, 'was_impossible': False}
user: 2.0        item: 2          r_ui = 2.88   est = 2.96   {'actual_k': 25, 'was_impossible': False}
user: 2.0        item: 3          r_ui = 3.64   est = 3.52   {'actual_k': 25, 'was_impossible': False}
user: 2.0        item: 4          r_ui = 3.80   est = 3.65   {'actual_k': 25, 'was_impossible': False}
user: 2.0        item: 5          r_ui = 3.88   est = 3.91   {'actual_k': 25, 'was_impossible': False}
user: 2.0        item: 6          r_ui = 3.88   est = 3.73   {'actual_k': 25, 'was_impossible': False}
user: 2.0        item: 7          r_ui = 2.56   est = 2.40   {'actual_k': 25, 'was_impossible': False}
user: 2.0        item: 8          r_ui = 3.64   est = 3.59   {'actual_k': 25, 'was_impossible': False}
user: 2.0        item: 9          r_ui = 3.64   est = 3.55   {'actual_k':

# rs

In [33]:
name_dict ={'김밥천국': 0,
 '포보스': 1,
 '삼백집': 2,
 '밥뜨랑': 3,
 '다원': 4,
 '한려수도': 5,
 '봉태민': 6,
 '종로김밥': 7,
 '명동칼국수': 8,
 '돈토': 9,
 '파리바게트': 10}

In [34]:
import pandas as pd
des6 = pd.read_csv('lunch_project1.csv', encoding='cp949')
des6['restaurant']=name_dict.keys()

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [36]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(des6['words'])
tfidf_matrix.shape

(11, 17400)

In [37]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sin = linear_kernel(tfidf_matrix, tfidf_matrix)

In [38]:
indices = pd.Series(des6.index, index = des6['restaurant'])

In [39]:
def rs(title, n=10, sim = cosine_sin):
    idx = indices[title] #해당 영화의 index를 찾아줌
    sim_score = list(enumerate(cosine_sin[idx])) # 순서대로 0,1,2..와 함께 해당 영화와 다른 영화간의 cosine_sin값이 튜플 형태로 들어감
    sim_score = sorted(sim_score, key = lambda x : x[1], reverse = True) # 값에 따라 정렬해줌, 내림차순 정렬
    sim_score = sim_score[1:(n+1)] # 자기 자신이 아닌(=1이므로 가장 큼) n개만큼 반환
    rest_index = [i[0] for i in sim_score] # (idx, cosin_sin값)[0] --> 해당 영화의 인덱스 값 반환
    return des6.restaurant.iloc[rest_index]

In [40]:
### 점수 확인용
def rs2(title, n=10, sim = cosine_sin):
    idx = indices[title] #해당 영화의 index를 찾아줌
    sim_score = list(enumerate(cosine_sin[idx])) # 순서대로 0,1,2..와 함께 해당 영화와 다른 영화간의 cosine_sin값이 튜플 형태로 들어감
    sim_score = sorted(sim_score, key = lambda x : x[1], reverse = True) # 값에 따라 정렬해줌, 내림차순 정렬
    sim_score = sim_score[1:(n+1)] # 자기 자신이 아닌(=1이므로 가장 큼) n개만큼 반환
    rest_index = [i[0] for i in sim_score] # (idx, cosin_sin값)[0] --> 해당 영화의 인덱스 값 반환
    des7 = pd.DataFrame(des6.restaurant.iloc[rest_index])
    des7['sim_score']=sim_score
    return des7

In [41]:
for i in name_dict.keys() :
    print(i)
    print(rs(i))
    print('\n')

김밥천국
7      종로김밥
2       삼백집
6       봉태민
8     명동칼국수
3       밥뜨랑
9        돈토
4        다원
5      한려수도
1       포보스
10    파리바게트
Name: restaurant, dtype: object


포보스
4        다원
5      한려수도
8     명동칼국수
9        돈토
6       봉태민
2       삼백집
3       밥뜨랑
0      김밥천국
7      종로김밥
10    파리바게트
Name: restaurant, dtype: object


삼백집
5      한려수도
4        다원
9        돈토
8     명동칼국수
6       봉태민
0      김밥천국
1       포보스
7      종로김밥
3       밥뜨랑
10    파리바게트
Name: restaurant, dtype: object


밥뜨랑
9        돈토
4        다원
5      한려수도
6       봉태민
8     명동칼국수
1       포보스
2       삼백집
0      김밥천국
7      종로김밥
10    파리바게트
Name: restaurant, dtype: object


다원
5      한려수도
1       포보스
9        돈토
8     명동칼국수
6       봉태민
3       밥뜨랑
2       삼백집
0      김밥천국
7      종로김밥
10    파리바게트
Name: restaurant, dtype: object


한려수도
4        다원
9        돈토
1       포보스
8     명동칼국수
2       삼백집
6       봉태민
3       밥뜨랑
0      김밥천국
7      종로김밥
10    파리바게트
Name: restaurant, dtype: object


봉태민
4        다원
9        돈토
5      한려수도
8     명동칼국수

# 결과 프린트 

In [42]:
user_no=0.0

from surprise import PredictionImpossible
ests=[]
for i in range(10):
    a= rating[i]
    pred = algo.predict(user_no, i, r_ui= a, verbose=True)
    ests.append(pred.est)

user: 0.0        item: 0          r_ui = 3.00   est = 2.63   {'actual_k': 25, 'was_impossible': False}
user: 0.0        item: 1          r_ui = 4.00   est = 3.52   {'actual_k': 25, 'was_impossible': False}
user: 0.0        item: 2          r_ui = 3.00   est = 2.95   {'actual_k': 25, 'was_impossible': False}
user: 0.0        item: 3          r_ui = 3.00   est = 3.44   {'actual_k': 25, 'was_impossible': False}
user: 0.0        item: 4          r_ui = 3.00   est = 3.68   {'actual_k': 25, 'was_impossible': False}
user: 0.0        item: 5          r_ui = 5.00   est = 4.08   {'actual_k': 25, 'was_impossible': False}
user: 0.0        item: 6          r_ui = 3.00   est = 3.75   {'actual_k': 25, 'was_impossible': False}
user: 0.0        item: 7          r_ui = 3.00   est = 2.68   {'actual_k': 25, 'was_impossible': False}
user: 0.0        item: 8          r_ui = 4.00   est = 3.76   {'actual_k': 25, 'was_impossible': False}
user: 0.0        item: 9          r_ui = 3.00   est = 3.54   {'actual_k':

In [43]:
a=ests.index(max(ests))
user_no=0.0

for rest, num in name_dict.items():
    if num == a:
        print(" user id :", user_no," 을 위한 추천 식당 : ", rest, "\n비슷한 식당 : \n",rs(a)[:2])


 user id : 0.0  을 위한 추천 식당 :  한려수도 
비슷한 식당 : 
 4    다원
9    돈토
Name: restaurant, dtype: object
